# Test Python Runtime Type Validation Libraries

## Test Functions
Let's first define a two type annotated test functions:

In [36]:
def is_larger_1(num_dict: dict[str, float]) -> dict[str, bool]:
    """Translates float values of the input dictionary into bool in the output dict
    depending on whether the value is > 1 or not."""
    try:
        return {key: num_dict[key] > 1 for key in num_dict}
    except:
        return {}


def is_larger_1_should_fail(num_dict: dict[str, float]) -> dict[str, bool]:
    """It should translates float values of the input dictionary into bool in the output dict
    depending on whether the value is > 1 or not. However, the output values are 1 if
    higher and 0 if lower or equal to 1. (This 1/0 representation could be coerced into
    True/False.)"""
    try:
        return {key: int(num_dict[key] > 1) for key in num_dict}
    except:
        return {}

Both functions are doing basically the same thing:

They accept a dictionary with str keys and float values and check for each value whether
it is larger 1. They are both intended to return a dictionary with boolean values indicating
the results.

However, the second function formats the output dict wrong. It is using 1/0 integers
instead of True/False for the values. Of course, the 1/0 encoding could be easily
coerced into a boolean representation, however, a strict type checker should throw an
exception instead.

## Test Data
Let's next define some test data that could be used with the above functions:

In [37]:
test_dict = {"a": 1.2, "b": 34.56}
test_dict_should_fail = {"a": "1.2", "b": "34.56"}
test_dict_should_definitely_fail = {"a": lambda x: x * 2, "b": None}

The first example is alright and complies with the type restrictions of the 
functions' input argument.

The second example uses int instead of float values. A smart parser could translate
int values into floats, however, a strict type checker should throw an exception.

The third option is definitely wrong as there is no intuitive way to translate the
given values into floats.

## Test Cases
Let's combine the test functions and test data into a couple of test cases:

In [38]:
from typing import Callable
import traceback

def run_test_cases(func_ok: Callable, func_fail: Callable) -> None:
    """Run test cases with provided functions."""

    print(">>> Case 1 (should pass):")
    try:
        print(func_ok(test_dict))
    except:
        print(traceback.format_exc())

    print("\n\n>>> Case 2 (should fail):")
    try:
        print(func_ok(test_dict_should_fail))
    except:
        print(traceback.format_exc())

    print("\n\n>>> Case 3 (should fail):")
    try:
        print(func_fail(test_dict))
    except:
        print(traceback.format_exc())

    print("\n\n>>> Case 4 (should fail):")
    try:
        print(func_ok(test_dict_should_definitely_fail))
    except:
        print(traceback.format_exc())


run_test_cases(is_larger_1, is_larger_1_should_fail)

>>> Case 1 (should pass):
{'a': True, 'b': True}


>>> Case 2 (should fail):
{}


>>> Case 3 (should fail):
{'a': 1, 'b': 1}


>>> Case 4 (should fail):
{}


Except for Case 1, all cases should fail. Either because the return value of the function
is not correct or the input test data is wrong.

Because we have not applied any type validation utility, yet, every test passes without
complains.

# Test with Pydantic Decorator
Now let's apply the `validate_arguments` decorator utility from the pydantic library:

In [39]:
from pydantic import validate_arguments

@validate_arguments
def pd_is_larger_1(num_dict: dict[str, float]) -> dict[str, bool]:
    """Translates float values of the input dictionary into bool in the output dict
    depending on whether the value is > 1 or not."""
    try:
        return {key: num_dict[key] > 1 for key in num_dict}
    except:
        return {}

@validate_arguments
def pd_is_larger_1_should_fail(num_dict: dict[str, float]) -> dict[str, bool]:
    """It should translates float values of the input dictionary into bool in the output dict
    depending on whether the value is > 1 or not. However, the output values are 1 if
    higher and 0 if lower or equal to 1. (This 1/0 representation could be coerced into
    True/False.)"""
    try:
        return {key: int(num_dict[key] > 1) for key in num_dict}
    except:
        return {}

Next, let's run the tests with the pydantic-validated functions:

In [40]:
run_test_cases(pd_is_larger_1, pd_is_larger_1_should_fail)

>>> Case 1 (should pass):
{'a': True, 'b': True}


>>> Case 2 (should fail):
{'a': True, 'b': True}


>>> Case 3 (should fail):
{'a': 1, 'b': 1}


>>> Case 4 (should fail):
Traceback (most recent call last):
  File "/tmp/ipykernel_15302/313806667.py", line 27, in run_test_cases
    print(func_ok(test_dict_should_definitely_fail))
  File "pydantic/decorator.py", line 40, in pydantic.decorator.validate_arguments.validate.wrapper_function
  File "pydantic/decorator.py", line 133, in pydantic.decorator.ValidatedFunction.call
  File "pydantic/decorator.py", line 130, in pydantic.decorator.ValidatedFunction.init_model_instance
  File "pydantic/main.py", line 331, in pydantic.main.BaseModel.__init__
pydantic.error_wrappers.ValidationError: 2 validation errors for PdIsLarger1
num_dict -> a
  value is not a valid float (type=type_error.float)
num_dict -> b
  none is not an allowed value (type=type_error.none.not_allowed)



As you can see, only the completely wrong test case is failing. The rest is passing and 
working just fine. The reason is pydantic is primarily a parsing library that tries to 
coerce input data into the expected type. Validation is only a side effect of that and
only happens on the coerced data.

## Test with Pydantic Decorator and Strict Types
If you want to use pydantic in strict mode, there is the option of using the 
pydantic-specific "Strict" typehints. For that we have to redefine the functions using
the special types: 

In [41]:

from pydantic import StrictStr, StrictFloat, StrictBool


@validate_arguments
def spd_is_larger_1(num_dict: dict[StrictStr, StrictFloat]) -> dict[StrictStr, StrictBool]:
    """Translates float values of the input dictionary into bool in the output dict
    depending on whether the value is > 1 or not."""
    try:
        return {key: num_dict[key] > 1 for key in num_dict}
    except:
        return {}


@validate_arguments
def spd_is_larger_1_should_fail(
    num_dict: dict[StrictStr, StrictFloat]
) -> dict[StrictStr, StrictBool]:
    """It should translates float values of the input dictionary into bool in the output dict
    depending on whether the value is > 1 or not. However, the output values are 1 if
    higher and 0 if lower or equal to 1. (This 1/0 representation could be coerced into
    True/False.)"""
    try:
        return {key: int(num_dict[key] > 1) for key in num_dict}
    except:
        return {}

<!--
 Copyright 2022 Universität Tübingen, DKFZ and EMBL
 for the German Human Genome-Phenome Archive (GHGA)
 
 Licensed under the Apache License, Version 2.0 (the "License");
 you may not use this file except in compliance with the License.
 You may obtain a copy of the License at
 
     http://www.apache.org/licenses/LICENSE-2.0
 
 Unless required by applicable law or agreed to in writing, software
 distributed under the License is distributed on an "AS IS" BASIS,
 WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
 See the License for the specific language governing permissions and
 limitations under the License.
-->



And run the tests again:

In [42]:
run_test_cases(spd_is_larger_1, spd_is_larger_1_should_fail)

>>> Case 1 (should pass):
{'a': True, 'b': True}


>>> Case 2 (should fail):
Traceback (most recent call last):
  File "/tmp/ipykernel_15302/313806667.py", line 15, in run_test_cases
    print(func_ok(test_dict_should_fail))
  File "pydantic/decorator.py", line 40, in pydantic.decorator.validate_arguments.validate.wrapper_function
  File "pydantic/decorator.py", line 133, in pydantic.decorator.ValidatedFunction.call
  File "pydantic/decorator.py", line 130, in pydantic.decorator.ValidatedFunction.init_model_instance
  File "pydantic/main.py", line 331, in pydantic.main.BaseModel.__init__
pydantic.error_wrappers.ValidationError: 2 validation errors for SpdIsLarger1
num_dict -> a
  value is not a valid float (type=type_error.float)
num_dict -> b
  value is not a valid float (type=type_error.float)



>>> Case 3 (should fail):
{'a': 1, 'b': 1}


>>> Case 4 (should fail):
Traceback (most recent call last):
  File "/tmp/ipykernel_15302/313806667.py", line 27, in run_test_cases
    print(fun

This was almost what we want. However, Case 3 is still not failing even though it should.
The reason is that the pydantic decorator only checks input arguments not the output
types.

## Test with Typeguard Decorator
Again we need to decorate our defined funtions with the utility provided by Typeguard:

In [43]:
from typeguard import typechecked

@typechecked
def tg_is_larger_1(num_dict: dict[str, float]) -> dict[str, bool]:
    """Translates float values of the input dictionary into bool in the output dict
    depending on whether the value is > 1 or not."""
    try:
        return {key: num_dict[key] > 1 for key in num_dict}
    except:
        return {}

@typechecked
def tg_is_larger_1_should_fail(num_dict: dict[str, float]) -> dict[str, bool]:
    """It should translates float values of the input dictionary into bool in the output dict
    depending on whether the value is > 1 or not. However, the output values are 1 if
    higher and 0 if lower or equal to 1. (This 1/0 representation could be coerced into
    True/False.)"""
    try:
        return {key: int(num_dict[key] > 1) for key in num_dict}
    except:
        return {}

Let's run the tests:

In [44]:
run_test_cases(tg_is_larger_1, tg_is_larger_1_should_fail)

>>> Case 1 (should pass):
{'a': True, 'b': True}


>>> Case 2 (should fail):
Traceback (most recent call last):
  File "/tmp/ipykernel_15302/313806667.py", line 15, in run_test_cases
    print(func_ok(test_dict_should_fail))
  File "/home/vscode/.local/lib/python3.9/site-packages/typeguard/__init__.py", line 1032, in wrapper
    check_argument_types(memo)
  File "/home/vscode/.local/lib/python3.9/site-packages/typeguard/__init__.py", line 875, in check_argument_types
    raise TypeError(*exc.args) from None
TypeError: type of argument "num_dict"['a'] must be either float or int; got str instead



>>> Case 3 (should fail):
Traceback (most recent call last):
  File "/tmp/ipykernel_15302/313806667.py", line 21, in run_test_cases
    print(func_fail(test_dict))
  File "/home/vscode/.local/lib/python3.9/site-packages/typeguard/__init__.py", line 1037, in wrapper
    raise TypeError(*exc.args) from None
TypeError: type of the return value['a'] must be bool; got int instead



>>> Case 4 (sh

This is exactly the behavior we want.